# ArchiCrop tutorial - Maize

This notebook is a tutorial for executing ArchiCrop model for maize.

## 0. Imports

In [1]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene

from openalea.archicrop.simulation import read_sti_file, read_xml_file
from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
%gui qt

## 1. Retrieve crop model parameters about management

Retrieve crop model parameters regarding the spatial (and temporal) configuration of the crop.

In [2]:
file_tec_xml = 'sorgho_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 0.4

## 2. Retrieve crop model growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [3]:
stics_output_file = 'mod_ssorghum.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [4]:
# TODO : verify the values in literature
nb_phy=13 # number of phytomers on the main stem : [10,40] (Ndiaye et al., 2021; Lafarge and Tardieu, 2002; Clerget, 2008; Ganeme et al., 2022)

# Stem
height=2*max(height_canopy) # potential plant height : [50,400] (Ganeme et al., 2022; Nebie et al., 2013)
stem_q=1.1 # parameter for ligule height distribution along axis : [1.1] (Kaitaniemi et al., 1999) 
diam_base=2.5 # stem base diameter : [2.2] (Ndiaye et al., 2021)
diam_top=1.5 # stem top diameter: [1.2] (Ndiaye et al., 2021)

# Leaf blade area 
leaf_area=2*max(leaf_area_plant) # potential plant leaf area : [] (Clerget, 2008)
rmax=0.9 # parameter for leaf area distribution along axis : [0.6,0.8] (Kaitaniemi et al., 1999; Welcker et al., )
skew=0.0005 # parameter for leaf area distribution along axis : [] (Kaitaniemi et al., 1999; Welcker et al., )
wl=0.12 # leaf blade width-to-length ratio : [0.1, 0.12] ()
klig=0.6 # parameter for leaf blade shape
swmax=0.55 # parameter for leaf blade shape
f1=0.64 # parameter for leaf blade shape
f2=0.92 # parameter for leaf blade shape

# Leaf blade position in space
insertion_angle=35 # leaf blade insertion angle : [10,50] (Truong et al., 2015; Kaitaniemi et al., 1999)
scurv=0.7 # leaf blade relative inflexion point : [0.6, 0.8] ()
curvature=120 # leaf blade insertion-to-tip angle : [45, 135] (Kaitaniemi et al., 1999)
phyllotactic_angle=180 # phyllotactic angle : [180] (Davis et al., 2024)
phyllotactic_deviation=10 # half-deviation to phyllotactic angle : [0,90] (Davis et al., 2024)

# Development
phyllochron=30 # phyllochron, i.e. stem element appearance rate : [40,65 then x1.6-2.5] (Clerget, 2008)
plastochron=40 # plastochron, i.e. leaf blade appearance rate : [34,46 then 80-93] (Rami Kumar et al., 2009)

# Senescence (from crop model parameters)
file_plt_xml = 'proto_sorghum_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase
leaf_lifespan=[lifespan_early, lifespan]

# Tillering
nb_tillers=0 # number of tillers : [0,6] (Lafarge et al., 2002)
tiller_delay=1 # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller : [] ()
reduction_factor=1 # reduction factor between tillers of consecutive order : [] ()

## 4. Instantiate ArchiCrop object

In [5]:
plant = ArchiCrop(height=height, 
                  nb_phy=nb_phy,
                  leaf_area=leaf_area,
                  wl=wl, diam_base=diam_base, diam_top=diam_top, 
                  insertion_angle=insertion_angle, scurv=scurv, curvature=curvature, 
                  klig=klig, swmax=swmax, f1=f1, f2=f2, 
                  stem_q=stem_q, rmax=rmax, skew=skew,
                  phyllotactic_angle=phyllotactic_angle,
                  phyllotactic_deviation=phyllotactic_deviation,
                  phyllochron=phyllochron, 
                  plastochron=plastochron, 
                  leaf_lifespan=leaf_lifespan,
                  nb_tillers=nb_tillers, tiller_delay=tiller_delay, reduction_factor=reduction_factor,
                  daily_dynamics=daily_dynamics)

## 5. Generate a potential plant 

Generate a potential plant from architectural parameters. 

This creates an MTG (Multi-scale Tree Graph, cf [doc](https://mtg.readthedocs.io/en/latest/user/tutorial.html)) as a property of the object plant.

In [6]:
plant.generate_potential_plant()

## 6. Simulate growth and senescence of the plant

Simulate growth and senescence of this plant following the daily dynamics.

In [7]:
growing_plant = plant.grow_plant()

## 7. Plot 3D scene

In [8]:
times = [t for i,t in enumerate(thermal_time) if i%5 == 0]
mean_time = sum(times) / len(times)
positions = [ (0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(list(growing_plant.values())) if i%5 == 0], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…